In [1]:
import numpy as np
import cv2 as cv
from pathlib import Path
import time

path = Path(".")
path = path.glob("airplanes/*.jpg")

count = 0
totalTime = 0

src = cv.cuda_GpuMat()

for imagepath in path:

    # GRAYSCALE
    src_host = cv.imread(str(imagepath), cv.IMREAD_GRAYSCALE)
    src.upload(src_host)

    start_t = time.time()
    
    gaussian_filter = cv.cuda.createGaussianFilter(src.type(), src.type(), (3, 3), 0, cv.BORDER_DEFAULT)
    gpu_blur = gaussian_filter.apply(src)

    kernel_x = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    kernel_y = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])

    img_prewittX = cv.cuda.createLinearFilter(gpu_blur.type(), gpu_blur.type(), kernel_x, (-1, -1))
    gpu_prewittX = img_prewittX.apply(gpu_blur)
    img_prewittY = cv.cuda.createLinearFilter(gpu_blur.type(), gpu_blur.type(), kernel_y, (-1, -1))
    gpu_prewittY = img_prewittY.apply(gpu_blur)

    # gpu_prewitt = cv.cuda.add(gpu_prewittX, gpu_prewittY)

    gpu_prewitt = cv.cuda.addWeighted(gpu_prewittX, 0.5, gpu_prewittY, 0.5, 0)
    
    totalTime = totalTime + (time.time() - start_t)
    
    result_host = gpu_prewitt.download()
    
    count += 1;
#     cv.imwrite("output/OP" + str(count) + ".jpg", result_host)

print("ImageCount =", count, "\nTimeRequired =", totalTime)


ImageCount = 800 
TimeRequired = 1.6519083976745605
